 The code in this notebook, **when run in order**, will:

#### (1) Produce a dataframe of tweets for a certain keyword (columns right now are, location and tweet text; can modify the search criteria as necessary (location lists, hashtags, etc.), saved as a csv

#### (2) Delete the duplicates/very similar spam tweets but leave one copy behind. 

#### (3) Clean the remaining tweets and saves as a csv

By the end of the notebook, you'll have a clean twitter dataframe that's ready for analysis 

In [1]:
import os
import tweepy
import pandas as pd
import time
import itertools
import numpy as np
from itertools import chain


In [2]:
consumer_key= '3emnsuDpqLBXlxD5UPUxVnxTt'
consumer_secret= 'ZPZBilbjbca7pYAmMv05hlNKsxLb4CJd6A5kNa2JNIs62BjF7m'
access_token= '1406213154-M77vYKXz7wIAUzzJkqLVzfDH0D6K11xMJkzGHzJ'
access_token_secret= 'iLcP4BuW4jWRYIHEiIZwsD5jgYlMY1aTmtSFwPQqbaQrX'

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

## Specify tweet search/filtering criteria here:

In [4]:
search_words = "\"mask free\" OR #maskfree OR maskfree"
date_since = "2021-03-10"
new_search = search_words + " -filter:retweets"


## Set location labels and corresponding location coordinates and radii here:

In [5]:
## ** In the order of the counties listed in the google doc, and put in the Facebook chat

locs_labels = ['Texas test',
'New York County NY',
 'County of San Francisco CA',
 'Hudson County NJ',
 'Suffolk County MA',
 'Philadelphia County PA',
 'Washington DC',
 'Alexandria VA',
 'Baltimore MD',
 'Cook County IL',
 'St. Louis County MO',
 'Milwaukee County WI',
 'Denver County CO',
 'Orange County CA',
 'Pinellas County FL',
 'Ramsey County MN',
 'Wayne County, MI',
 'Cuyahoga County OH',
 'Dallas County TX',
 'DeKalb County GA',
 'Salt Lake County UT',
 'Marion County IN',
 'Jefferson County KY',
 'Johnson County KS',
 'King County WA']


locs = ['29.88548725,-96.27736949,100mi',
    '40.7741618671,-73.9697971209,100mi',
    '37.7597150131,-122.693975573,100mi',
    '40.7309049975,-74.0759554422,100mi',
    '42.3486893925,-70.9856096697,100mi',
    '40.0076360466,-75.1339446794,100mi', 
    '38.9047737146,-77.0163026702,100mi',
    '38.8184535854,-77.0862060584,100mi',
    '39.443072852,-76.6163229502,100mi',
    '41.8954290221,-87.6461407931,100mi',
    '38.6406885662,-90.4433975603,100mi',
    '43.0154568883,-87.5805486607,100mi',
    '39.7621646287,-104.875801423,100mi',
    '33.6769134328,-117.776142822,100mi',
    '27.9026762041,-82.7395539464,100mi',
    '45.0170541432,-93.099606274,100mi',
    '42.2847454761,-83.2611328686,100mi',
    '41.6350590585,-81.7001959208,100mi',
    '32.7666300195,-96.7778770663,100mi',
    '33.7715468713,-84.226432416,100mi',
    '40.6673139262,-111.923601307,100mi',
    '39.7817096805,-86.1384716709,100mi',
    '38.1871910002,-85.6591577058,100mi',
    '38.8837685125,-94.8222610425,100mi',
    '47.4909231926,-121.83595173,100mi']

## Just zip locs_labels with locs and you'd have the coordinates mapped to their names

mapped_locations = list(zip(locs_labels, locs))

In [6]:
mapped_locations

[('Texas test', '29.88548725,-96.27736949,100mi'),
 ('New York County NY', '40.7741618671,-73.9697971209,100mi'),
 ('County of San Francisco CA', '37.7597150131,-122.693975573,100mi'),
 ('Hudson County NJ', '40.7309049975,-74.0759554422,100mi'),
 ('Suffolk County MA', '42.3486893925,-70.9856096697,100mi'),
 ('Philadelphia County PA', '40.0076360466,-75.1339446794,100mi'),
 ('Washington DC', '38.9047737146,-77.0163026702,100mi'),
 ('Alexandria VA', '38.8184535854,-77.0862060584,100mi'),
 ('Baltimore MD', '39.443072852,-76.6163229502,100mi'),
 ('Cook County IL', '41.8954290221,-87.6461407931,100mi'),
 ('St. Louis County MO', '38.6406885662,-90.4433975603,100mi'),
 ('Milwaukee County WI', '43.0154568883,-87.5805486607,100mi'),
 ('Denver County CO', '39.7621646287,-104.875801423,100mi'),
 ('Orange County CA', '33.6769134328,-117.776142822,100mi'),
 ('Pinellas County FL', '27.9026762041,-82.7395539464,100mi'),
 ('Ramsey County MN', '45.0170541432,-93.099606274,100mi'),
 ('Wayne County, MI',

In [7]:
len(locs) # ensure that locs and loc_labels lens match; should have through zip funct anyway

25

In [8]:
len(locs_labels)

25

In [319]:
'''
# Nassau county, NY
tweet_lst=[]
geoc="40.6546,73.5594,50mi"
txgeoc = '29.88548725,-96.27736949,100mi'
migeoc = '25.6112362,-80.55170587,50mi'
for tweet in tweepy.Cursor(api.search,q='#maskfree', lang="en", since=date_since, geocode=txgeoc, tweet_mode='extended').items(1000):
    print(tweet)
    if(tweet.coordinates != None):
        print('we got here')
        tweet_lst.append([tweet.id,tweet.coordinates[0],
                          tweet.coordinates[1],
                          tweet.user.screen_name,
                          tweet.user.name, tweet.text])
    else:
        print("it was none")
tweet_df = pd.DataFrame(tweet_lst, columns=['id', 'lat','long','username', 'name', 'tweet'])
'''

'\n# Nassau county, NY\ntweet_lst=[]\ngeoc="40.6546,73.5594,50mi"\ntxgeoc = \'29.88548725,-96.27736949,100mi\'\nmigeoc = \'25.6112362,-80.55170587,50mi\'\nfor tweet in tweepy.Cursor(api.search,q=\'#maskfree\', lang="en", since=date_since, geocode=txgeoc, tweet_mode=\'extended\').items(1000):\n    print(tweet)\n    if(tweet.coordinates != None):\n        print(\'we got here\')\n        tweet_lst.append([tweet.id,tweet.coordinates[0],\n                          tweet.coordinates[1],\n                          tweet.user.screen_name,\n                          tweet.user.name, tweet.text])\n    else:\n        print("it was none")\ntweet_df = pd.DataFrame(tweet_lst, columns=[\'id\', \'lat\',\'long\',\'username\', \'name\', \'tweet\'])\n'

In [320]:
#tweet_df

In [321]:
'''
### TEXAS TEST ####


tweet_lst=[]
geoc="40.6546,73.5594,50mi"
txgeoc = '29.88548725,-96.27736949,100mi'
migeoc = '25.6112362,-80.55170587,50mi'

    

# Creation of query method using parameters
try:
    tweets = tweepy.Cursor(api.search,q='mask free', lang="en", since=date_since, geocode=txgeoc, tweet_mode='extended').items(5) 
# Pulling information from tweets iterable object
    tweets_list = [[tweet.created_at, tweet.id, tweet.full_text] for tweet in tweets]
    tweets_sort = tweets_list.sort()
    no_dupes = list(k for k,_ in itertools.groupby(tweets_sort))
    print(tweets_list)
 # Creation of dataframe from tweets list
 # Add or remove columns as you remove tweet information
    tweets_d = pd.DataFrame(no_dupes)

except BaseException as e:
    print('failed on_status,',str(e))

tweets_f = pd.DataFrame(no_dupes)
'''

'\n### TEXAS TEST ####\n\n\ntweet_lst=[]\ngeoc="40.6546,73.5594,50mi"\ntxgeoc = \'29.88548725,-96.27736949,100mi\'\nmigeoc = \'25.6112362,-80.55170587,50mi\'\n\n    \n\n# Creation of query method using parameters\ntry:\n    tweets = tweepy.Cursor(api.search,q=\'mask free\', lang="en", since=date_since, geocode=txgeoc, tweet_mode=\'extended\').items(5) \n# Pulling information from tweets iterable object\n    tweets_list = [[tweet.created_at, tweet.id, tweet.full_text] for tweet in tweets]\n    tweets_sort = tweets_list.sort()\n    no_dupes = list(k for k,_ in itertools.groupby(tweets_sort))\n    print(tweets_list)\n # Creation of dataframe from tweets list\n # Add or remove columns as you remove tweet information\n    tweets_d = pd.DataFrame(no_dupes)\n\nexcept BaseException as e:\n    print(\'failed on_status,\',str(e))\n\ntweets_f = pd.DataFrame(no_dupes)\n'

## The code that does the actual scraping by location:

In [19]:
# SCRAPING TWEETS FOR ALL LOCATIONS # #Kaylah added date info here

tweets_list = []
for loc in mapped_locations:
    try:
        tweets = tweepy.Cursor(api.search,q=new_search, lang="en", since=date_since, geocode=loc[1], tweet_mode='extended').items(1000) 
        tweets_list.append((loc[0], [[tweet.full_text, tweet.created_at.strftime("%d-%b-%Y")] for tweet in tweets]))
        print(tweets_list)
        tweets_df = pd.DataFrame(tweets_list)

    except BaseException as e:
        print('failed on_status,',str(e))
        time.sleep(3)
    

[('Texas test', [['@tigerfan1975lsu @AmericanAir Thanks @AmericanAir ! Anyway we can get a badge or something to make sure everybody knows we ride with Kerry mask free all flight long? Please give us an idea of what it may look like so we know it’s authentic. Maybe add a barcode for scanning, just a thought.', '18-Mar-2021'], ['What are some of the top compliance &amp; ethics stories from the past week? In the only weekly wrap up of all things compliance &amp; ethics @FCPA_Monitor  @tfoxlaw  review some of the top stories on This Week in FCPA, the Mask Free Texas edition.  https://t.co/8RJsnOoTOx @csuiteradio https://t.co/eCl2AajWH8', '17-Mar-2021'], ['What are some of the top compliance &amp; ethics stories from the past week? In the only weekly wrap up of all things compliance &amp; ethics @FCPA_Monitor  @tfoxlaw  review some of the top stories on This Week in FCPA, the Mask Free Texas edition.  https://t.co/8RJsnO7ipX @csuiteradio https://t.co/JrqHDgKWHG', '16-Mar-2021'], ['@ITHERET

[('Texas test', [['@tigerfan1975lsu @AmericanAir Thanks @AmericanAir ! Anyway we can get a badge or something to make sure everybody knows we ride with Kerry mask free all flight long? Please give us an idea of what it may look like so we know it’s authentic. Maybe add a barcode for scanning, just a thought.', '18-Mar-2021'], ['What are some of the top compliance &amp; ethics stories from the past week? In the only weekly wrap up of all things compliance &amp; ethics @FCPA_Monitor  @tfoxlaw  review some of the top stories on This Week in FCPA, the Mask Free Texas edition.  https://t.co/8RJsnOoTOx @csuiteradio https://t.co/eCl2AajWH8', '17-Mar-2021'], ['What are some of the top compliance &amp; ethics stories from the past week? In the only weekly wrap up of all things compliance &amp; ethics @FCPA_Monitor  @tfoxlaw  review some of the top stories on This Week in FCPA, the Mask Free Texas edition.  https://t.co/8RJsnO7ipX @csuiteradio https://t.co/JrqHDgKWHG', '16-Mar-2021'], ['@ITHERET

[('Texas test', [['@tigerfan1975lsu @AmericanAir Thanks @AmericanAir ! Anyway we can get a badge or something to make sure everybody knows we ride with Kerry mask free all flight long? Please give us an idea of what it may look like so we know it’s authentic. Maybe add a barcode for scanning, just a thought.', '18-Mar-2021'], ['What are some of the top compliance &amp; ethics stories from the past week? In the only weekly wrap up of all things compliance &amp; ethics @FCPA_Monitor  @tfoxlaw  review some of the top stories on This Week in FCPA, the Mask Free Texas edition.  https://t.co/8RJsnOoTOx @csuiteradio https://t.co/eCl2AajWH8', '17-Mar-2021'], ['What are some of the top compliance &amp; ethics stories from the past week? In the only weekly wrap up of all things compliance &amp; ethics @FCPA_Monitor  @tfoxlaw  review some of the top stories on This Week in FCPA, the Mask Free Texas edition.  https://t.co/8RJsnO7ipX @csuiteradio https://t.co/JrqHDgKWHG', '16-Mar-2021'], ['@ITHERET

[('Texas test', [['@tigerfan1975lsu @AmericanAir Thanks @AmericanAir ! Anyway we can get a badge or something to make sure everybody knows we ride with Kerry mask free all flight long? Please give us an idea of what it may look like so we know it’s authentic. Maybe add a barcode for scanning, just a thought.', '18-Mar-2021'], ['What are some of the top compliance &amp; ethics stories from the past week? In the only weekly wrap up of all things compliance &amp; ethics @FCPA_Monitor  @tfoxlaw  review some of the top stories on This Week in FCPA, the Mask Free Texas edition.  https://t.co/8RJsnOoTOx @csuiteradio https://t.co/eCl2AajWH8', '17-Mar-2021'], ['What are some of the top compliance &amp; ethics stories from the past week? In the only weekly wrap up of all things compliance &amp; ethics @FCPA_Monitor  @tfoxlaw  review some of the top stories on This Week in FCPA, the Mask Free Texas edition.  https://t.co/8RJsnO7ipX @csuiteradio https://t.co/JrqHDgKWHG', '16-Mar-2021'], ['@ITHERET

[('Texas test', [['@tigerfan1975lsu @AmericanAir Thanks @AmericanAir ! Anyway we can get a badge or something to make sure everybody knows we ride with Kerry mask free all flight long? Please give us an idea of what it may look like so we know it’s authentic. Maybe add a barcode for scanning, just a thought.', '18-Mar-2021'], ['What are some of the top compliance &amp; ethics stories from the past week? In the only weekly wrap up of all things compliance &amp; ethics @FCPA_Monitor  @tfoxlaw  review some of the top stories on This Week in FCPA, the Mask Free Texas edition.  https://t.co/8RJsnOoTOx @csuiteradio https://t.co/eCl2AajWH8', '17-Mar-2021'], ['What are some of the top compliance &amp; ethics stories from the past week? In the only weekly wrap up of all things compliance &amp; ethics @FCPA_Monitor  @tfoxlaw  review some of the top stories on This Week in FCPA, the Mask Free Texas edition.  https://t.co/8RJsnO7ipX @csuiteradio https://t.co/JrqHDgKWHG', '16-Mar-2021'], ['@ITHERET

...

[('Texas test', [['@tigerfan1975lsu @AmericanAir Thanks @AmericanAir ! Anyway we can get a badge or something to make sure everybody knows we ride with Kerry mask free all flight long? Please give us an idea of what it may look like so we know it’s authentic. Maybe add a barcode for scanning, just a thought.', '18-Mar-2021'], ['What are some of the top compliance &amp; ethics stories from the past week? In the only weekly wrap up of all things compliance &amp; ethics @FCPA_Monitor  @tfoxlaw  review some of the top stories on This Week in FCPA, the Mask Free Texas edition.  https://t.co/8RJsnOoTOx @csuiteradio https://t.co/eCl2AajWH8', '17-Mar-2021'], ['What are some of the top compliance &amp; ethics stories from the past week? In the only weekly wrap up of all things compliance &amp; ethics @FCPA_Monitor  @tfoxlaw  review some of the top stories on This Week in FCPA, the Mask Free Texas edition.  https://t.co/8RJsnO7ipX @csuiteradio https://t.co/JrqHDgKWHG', '16-Mar-2021'], ['@ITHERET

[('Texas test', [['@tigerfan1975lsu @AmericanAir Thanks @AmericanAir ! Anyway we can get a badge or something to make sure everybody knows we ride with Kerry mask free all flight long? Please give us an idea of what it may look like so we know it’s authentic. Maybe add a barcode for scanning, just a thought.', '18-Mar-2021'], ['What are some of the top compliance &amp; ethics stories from the past week? In the only weekly wrap up of all things compliance &amp; ethics @FCPA_Monitor  @tfoxlaw  review some of the top stories on This Week in FCPA, the Mask Free Texas edition.  https://t.co/8RJsnOoTOx @csuiteradio https://t.co/eCl2AajWH8', '17-Mar-2021'], ['What are some of the top compliance &amp; ethics stories from the past week? In the only weekly wrap up of all things compliance &amp; ethics @FCPA_Monitor  @tfoxlaw  review some of the top stories on This Week in FCPA, the Mask Free Texas edition.  https://t.co/8RJsnO7ipX @csuiteradio https://t.co/JrqHDgKWHG', '16-Mar-2021'], ['@ITHERET

[('Texas test', [['@tigerfan1975lsu @AmericanAir Thanks @AmericanAir ! Anyway we can get a badge or something to make sure everybody knows we ride with Kerry mask free all flight long? Please give us an idea of what it may look like so we know it’s authentic. Maybe add a barcode for scanning, just a thought.', '18-Mar-2021'], ['What are some of the top compliance &amp; ethics stories from the past week? In the only weekly wrap up of all things compliance &amp; ethics @FCPA_Monitor  @tfoxlaw  review some of the top stories on This Week in FCPA, the Mask Free Texas edition.  https://t.co/8RJsnOoTOx @csuiteradio https://t.co/eCl2AajWH8', '17-Mar-2021'], ['What are some of the top compliance &amp; ethics stories from the past week? In the only weekly wrap up of all things compliance &amp; ethics @FCPA_Monitor  @tfoxlaw  review some of the top stories on This Week in FCPA, the Mask Free Texas edition.  https://t.co/8RJsnO7ipX @csuiteradio https://t.co/JrqHDgKWHG', '16-Mar-2021'], ['@ITHERET

[('Texas test', [['@tigerfan1975lsu @AmericanAir Thanks @AmericanAir ! Anyway we can get a badge or something to make sure everybody knows we ride with Kerry mask free all flight long? Please give us an idea of what it may look like so we know it’s authentic. Maybe add a barcode for scanning, just a thought.', '18-Mar-2021'], ['What are some of the top compliance &amp; ethics stories from the past week? In the only weekly wrap up of all things compliance &amp; ethics @FCPA_Monitor  @tfoxlaw  review some of the top stories on This Week in FCPA, the Mask Free Texas edition.  https://t.co/8RJsnOoTOx @csuiteradio https://t.co/eCl2AajWH8', '17-Mar-2021'], ['What are some of the top compliance &amp; ethics stories from the past week? In the only weekly wrap up of all things compliance &amp; ethics @FCPA_Monitor  @tfoxlaw  review some of the top stories on This Week in FCPA, the Mask Free Texas edition.  https://t.co/8RJsnO7ipX @csuiteradio https://t.co/JrqHDgKWHG', '16-Mar-2021'], ['@ITHERET

[('Texas test', [['@tigerfan1975lsu @AmericanAir Thanks @AmericanAir ! Anyway we can get a badge or something to make sure everybody knows we ride with Kerry mask free all flight long? Please give us an idea of what it may look like so we know it’s authentic. Maybe add a barcode for scanning, just a thought.', '18-Mar-2021'], ['What are some of the top compliance &amp; ethics stories from the past week? In the only weekly wrap up of all things compliance &amp; ethics @FCPA_Monitor  @tfoxlaw  review some of the top stories on This Week in FCPA, the Mask Free Texas edition.  https://t.co/8RJsnOoTOx @csuiteradio https://t.co/eCl2AajWH8', '17-Mar-2021'], ['What are some of the top compliance &amp; ethics stories from the past week? In the only weekly wrap up of all things compliance &amp; ethics @FCPA_Monitor  @tfoxlaw  review some of the top stories on This Week in FCPA, the Mask Free Texas edition.  https://t.co/8RJsnO7ipX @csuiteradio https://t.co/JrqHDgKWHG', '16-Mar-2021'], ['@ITHERET

In [20]:
tweets_df

,0,1
0,Texas test,[[@tigerfan1975lsu @AmericanAir Thanks @Americ...
1,New York County NY,[[Y’all: Lockdowns don’t work\n\nMe: *Looks at...
2,County of San Francisco CA,[[“Each of us have our own timelines to normal...
3,Hudson County NJ,[[Y’all: Lockdowns don’t work\n\nMe: *Looks at...
4,Suffolk County MA,[[Looking to learn more about the Precision Fl...
5,Philadelphia County PA,[[Y’all: Lockdowns don’t work\n\nMe: *Looks at...
6,Washington DC,[[So even though they can still spread the dea...
7,Alexandria VA,[[So even though they can still spread the dea...
8,Baltimore MD,[[So even though they can still spread the dea...
9,Cook County IL,[[Fully vaccinated but scared to remove your m...


In [21]:
tweets_df.to_csv('tweet7_dataset.csv') # saves the dataframe


################################################################################################

# Tweet df loading and cleaning

In [22]:
import os
import tweepy
import pandas as pd
import time
import itertools
import numpy as np
from itertools import chain
import regex as re
import unicodedata
import nltk

## Loads tweet dataframe, ensures datatypes are appropriate, flattens tweets into one list for ease, labels columns

In [24]:
tweets_df = pd.read_csv('tweet7_dataset.csv', converters={'1': eval}, index_col=0)
tweets_df['tweets_list'] = tweets_df['1'].apply(lambda x : np.array(x).flatten())
tweets_df.columns = ['county', 'original_tweet', 'tweets_list']
tweets_df

,county,original_tweet,tweets_list
0,Texas test,[[@tigerfan1975lsu @AmericanAir Thanks @Americ...,[@tigerfan1975lsu @AmericanAir Thanks @America...
1,New York County NY,[[Y’all: Lockdowns don’t work\n\nMe: *Looks at...,[Y’all: Lockdowns don’t work\n\nMe: *Looks at ...
2,County of San Francisco CA,[[“Each of us have our own timelines to normal...,[“Each of us have our own timelines to normal....
3,Hudson County NJ,[[Y’all: Lockdowns don’t work\n\nMe: *Looks at...,[Y’all: Lockdowns don’t work\n\nMe: *Looks at ...
4,Suffolk County MA,[[Looking to learn more about the Precision Fl...,[Looking to learn more about the Precision Flo...
5,Philadelphia County PA,[[Y’all: Lockdowns don’t work\n\nMe: *Looks at...,[Y’all: Lockdowns don’t work\n\nMe: *Looks at ...
6,Washington DC,[[So even though they can still spread the dea...,[So even though they can still spread the dead...
7,Alexandria VA,[[So even though they can still spread the dea...,[So even though they can still spread the dead...
8,Baltimore MD,[[So even though they can still spread the dea...,[So even though they can still spread the dead...
9,Cook County IL,[[Fully vaccinated but scared to remove your m...,[Fully vaccinated but scared to remove your ma...


In [327]:
# texas  --- 226 total tweets
#for index, tweet in enumerate(list(tweets_df['tweets_list'].iloc[0])):
#    print(index, tweet)
    

## Now, expands dataset so there is one row per tweet (before, all tweets for a county were in one row)


In [29]:
t_explode = tweets_df.explode('original_tweet')
t_explode

,county,original_tweet,tweets_list
0,Texas test,[@tigerfan1975lsu @AmericanAir Thanks @America...,[@tigerfan1975lsu @AmericanAir Thanks @America...
0,Texas test,[What are some of the top compliance &amp; eth...,[@tigerfan1975lsu @AmericanAir Thanks @America...
0,Texas test,[What are some of the top compliance &amp; eth...,[@tigerfan1975lsu @AmericanAir Thanks @America...
0,Texas test,[@ITHERETWEETER1 @katyekellyeand1 @MySwilly @S...,[@tigerfan1975lsu @AmericanAir Thanks @America...
0,Texas test,[@DonaldJTrumpJr Please thank your dad for me ...,[@tigerfan1975lsu @AmericanAir Thanks @America...
...,...,...,...
23,Johnson County KS,[@thomaskaine5 It covered what needed to be co...,[And not a mask in sight....#PuntaCana #MaskFr...
24,King County WA,[@Hilton @HiltonHotels I see Hilton properties...,[@Hilton @HiltonHotels I see Hilton properties...
24,King County WA,[I never say anything to my cousin anymore bc ...,[@Hilton @HiltonHotels I see Hilton properties...
24,King County WA,[@offgridteacher I’ve eaten indoors but the ma...,[@Hilton @HiltonHotels I see Hilton properties...


In [76]:
# Created new column for date
for tweet_ind, tweet_row, in t_explode.iterrows():
    tweet_row['date_tweeted'] = tweet_row['original_tweet'][1]
    
# cleaning date layout for easier matching to COVID-19 dataframe

for date_ind, date_row in t_explode.iterrows():
    date_alt = date_row['date_tweeted'].replace('-', ' ')
    date_alt = date_alt.split()
    date_alt[1] = '3'
    date_alt[2] = '21'
    date_alt[0], date_alt[1] = date_alt[1], date_alt[0]
    date_alt = "/".join(date_alt)
    date_row['date_tweeted'] = date_alt
t_explode.head()
    

,county,original_tweet,tweets_list,date_tweeted,tweet_str,tweet_slice
0,Texas test,[@tigerfan1975lsu @AmericanAir Thanks @America...,[@tigerfan1975lsu @AmericanAir Thanks @America...,3/18/21,['@tigerfan1975lsu @AmericanAir Thanks @Americ...,r ! Anyway we can get a badge or something to ...
0,Texas test,[What are some of the top compliance &amp; eth...,[@tigerfan1975lsu @AmericanAir Thanks @America...,3/17/21,['What are some of the top compliance &amp; et...,stories from the past week? In the only weekl...
0,Texas test,[What are some of the top compliance &amp; eth...,[@tigerfan1975lsu @AmericanAir Thanks @America...,3/16/21,['What are some of the top compliance &amp; et...,stories from the past week? In the only weekl...
0,Texas test,[@ITHERETWEETER1 @katyekellyeand1 @MySwilly @S...,[@tigerfan1975lsu @AmericanAir Thanks @America...,3/16/21,['@ITHERETWEETER1 @katyekellyeand1 @MySwilly @...,KLORE1 @hellsoasis @DitseaYella @Promethiumban...
0,Texas test,[@DonaldJTrumpJr Please thank your dad for me ...,[@tigerfan1975lsu @AmericanAir Thanks @America...,3/16/21,"[""@DonaldJTrumpJr Please thank your dad for me...",ce he is not here on twitter! Being a retired ...


In [329]:
# tex = t_explode.loc[t_explode['county'] == 'Texas test']

# print(len(tex)) # confirm that this is 226; it is

In [77]:
t_explode['original_tweet'].iloc[56]

['Sale out pon the Astra zeneca sale out on fizer  1 fi 3 grand 2 fi 5 grand. Sale out buy 2 get a face mask free https://t.co/8kkX3Ai3gC',
 '14-Mar-2021']

***********
# See note:

I tried getting rid of duplicates a few ways; doing pairwise levenshtein distance comparisons between all the tweets took way too long, and creating a mask to then drop duplicate tweets was also complicated. I noticed that most very-similar-basically-duplicate tweets had the same text, but different handles and urls. So I took a slice of part of the text that was very likely to be shared between the similar tweets, created a new column in the dataset where each tweet-row got a corresponding tweet-slice, and then I deleted the rows where there were duplicate tweet-slices. As you can see, this resulted in a *lot* of data loss (3000+ tweets down to 850; see twitter_drop a few cells down), because many of them were duplicate/spam tweets.
***********

In [78]:
# converting to string data type 
t_explode["tweet_str"]= t_explode["original_tweet"].astype(str) 
  
# slicing till 2nd last element 
t_explode["tweet_slice"] = t_explode["tweet_str"].str.slice(50,1000) 
  
# display 
t_explode

,county,original_tweet,tweets_list,date_tweeted,tweet_str,tweet_slice
0,Texas test,[@tigerfan1975lsu @AmericanAir Thanks @America...,[@tigerfan1975lsu @AmericanAir Thanks @America...,3/18/21,['@tigerfan1975lsu @AmericanAir Thanks @Americ...,r ! Anyway we can get a badge or something to ...
0,Texas test,[What are some of the top compliance &amp; eth...,[@tigerfan1975lsu @AmericanAir Thanks @America...,3/17/21,['What are some of the top compliance &amp; et...,stories from the past week? In the only weekl...
0,Texas test,[What are some of the top compliance &amp; eth...,[@tigerfan1975lsu @AmericanAir Thanks @America...,3/16/21,['What are some of the top compliance &amp; et...,stories from the past week? In the only weekl...
0,Texas test,[@ITHERETWEETER1 @katyekellyeand1 @MySwilly @S...,[@tigerfan1975lsu @AmericanAir Thanks @America...,3/16/21,['@ITHERETWEETER1 @katyekellyeand1 @MySwilly @...,KLORE1 @hellsoasis @DitseaYella @Promethiumban...
0,Texas test,[@DonaldJTrumpJr Please thank your dad for me ...,[@tigerfan1975lsu @AmericanAir Thanks @America...,3/16/21,"[""@DonaldJTrumpJr Please thank your dad for me...",ce he is not here on twitter! Being a retired ...
...,...,...,...,...,...,...
23,Johnson County KS,[@thomaskaine5 It covered what needed to be co...,[And not a mask in sight....#PuntaCana #MaskFr...,3/12/21,['@thomaskaine5 It covered what needed to be c...,"ed. Sadly, I feel like he is preaching to the..."
24,King County WA,[@Hilton @HiltonHotels I see Hilton properties...,[@Hilton @HiltonHotels I see Hilton properties...,3/18/21,['@Hilton @HiltonHotels I see Hilton propertie...,Idaho requiring guests to wear masks while Ma...
24,King County WA,[I never say anything to my cousin anymore bc ...,[@Hilton @HiltonHotels I see Hilton properties...,3/16/21,['I never say anything to my cousin anymore bc...,is the only place she hasn’t blocked me and I ...
24,King County WA,[@offgridteacher I’ve eaten indoors but the ma...,[@Hilton @HiltonHotels I see Hilton properties...,3/13/21,['@offgridteacher I’ve eaten indoors but the m...,requirements are pretty strict (masks to enter...


### Can look through slice list to ensure that the slices look right

In [79]:
slice_list = t_explode['tweet_slice'].tolist()


for i, v in enumerate(slice_list):
    print(i,v)
#slice_list[184]

0 r ! Anyway we can get a badge or something to make sure everybody knows we ride with Kerry mask free all flight long? Please give us an idea of what it may look like so we know it’s authentic. Maybe add a barcode for scanning, just a thought.', '18-Mar-2021']
1  stories from the past week? In the only weekly wrap up of all things compliance &amp; ethics @FCPA_Monitor  @tfoxlaw  review some of the top stories on This Week in FCPA, the Mask Free Texas edition.  https://t.co/8RJsnOoTOx @csuiteradio https://t.co/eCl2AajWH8', '17-Mar-2021']
2  stories from the past week? In the only weekly wrap up of all things compliance &amp; ethics @FCPA_Monitor  @tfoxlaw  review some of the top stories on This Week in FCPA, the Mask Free Texas edition.  https://t.co/8RJsnO7ipX @csuiteradio https://t.co/JrqHDgKWHG', '16-Mar-2021']
3 KLORE1 @hellsoasis @DitseaYella @Promethiumband @HardlandRocks @ManeatGrass @KaiserCreation @sleepingisalice @CounterIntell14 @SilverbirdRock @CranberryMerch2 @A_and_D_Arch

In [80]:
twitter_drop = t_explode.drop_duplicates(subset=['tweet_slice'], keep='last')
twitter_drop

,county,original_tweet,tweets_list,date_tweeted,tweet_str,tweet_slice
0,Texas test,[@tigerfan1975lsu @AmericanAir Thanks @America...,[@tigerfan1975lsu @AmericanAir Thanks @America...,3/18/21,['@tigerfan1975lsu @AmericanAir Thanks @Americ...,r ! Anyway we can get a badge or something to ...
0,Texas test,[What are some of the top compliance &amp; eth...,[@tigerfan1975lsu @AmericanAir Thanks @America...,3/17/21,['What are some of the top compliance &amp; et...,stories from the past week? In the only weekl...
0,Texas test,[What are some of the top compliance &amp; eth...,[@tigerfan1975lsu @AmericanAir Thanks @America...,3/16/21,['What are some of the top compliance &amp; et...,stories from the past week? In the only weekl...
0,Texas test,[@ITHERETWEETER1 @katyekellyeand1 @MySwilly @S...,[@tigerfan1975lsu @AmericanAir Thanks @America...,3/16/21,['@ITHERETWEETER1 @katyekellyeand1 @MySwilly @...,KLORE1 @hellsoasis @DitseaYella @Promethiumban...
0,Texas test,[@DonaldJTrumpJr Please thank your dad for me ...,[@tigerfan1975lsu @AmericanAir Thanks @America...,3/16/21,"[""@DonaldJTrumpJr Please thank your dad for me...",ce he is not here on twitter! Being a retired ...
...,...,...,...,...,...,...
23,Johnson County KS,[@thomaskaine5 It covered what needed to be co...,[And not a mask in sight....#PuntaCana #MaskFr...,3/12/21,['@thomaskaine5 It covered what needed to be c...,"ed. Sadly, I feel like he is preaching to the..."
24,King County WA,[@Hilton @HiltonHotels I see Hilton properties...,[@Hilton @HiltonHotels I see Hilton properties...,3/18/21,['@Hilton @HiltonHotels I see Hilton propertie...,Idaho requiring guests to wear masks while Ma...
24,King County WA,[I never say anything to my cousin anymore bc ...,[@Hilton @HiltonHotels I see Hilton properties...,3/16/21,['I never say anything to my cousin anymore bc...,is the only place she hasn’t blocked me and I ...
24,King County WA,[@offgridteacher I’ve eaten indoors but the ma...,[@Hilton @HiltonHotels I see Hilton properties...,3/13/21,['@offgridteacher I’ve eaten indoors but the m...,requirements are pretty strict (masks to enter...


In [81]:
twitter_drop['tweet_str'].iloc[89]


'["So by all reports and the CDC NYC should be able to be fully vaccinated by summer. And by the current CDC guidelines just issued : f everyone present at an event has been fully vaccinated they can safely be together, mask free even. What are people\'s thoughts on this?", \'11-Mar-2021\']'

In [82]:
twitter_drop['tweet_str'].iloc[88]


'["@DailyCaller Sleepy Joe\'s 1st mistake was telling a bunch of free thinking adult Americans they can\'t do something. Does he really think no one left their house or wore masks. He just opened the flood gates for massive 4th parties. I know I\'ll be at quite a few different ones &amp; mask free", \'12-Mar-2021\']'

## Cleans the tweets

In [83]:
df_clean = twitter_drop

In [84]:
# lower case
df_clean['tweet_text'] = df_clean['tweet_str'].str.lower()

# remove url links
df_clean['tweet_text'] = df_clean['tweet_text'].apply(lambda x: re.sub(r'https?:\/\/\S+', '', x))

# remove url/website that didn't use http, is only checking for .com websites 
# so words that are seperated by a . are not removed
df_clean['tweet_text'] = df_clean['tweet_text'].apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', x))
# remove @mention
df_clean['tweet_text'] = df_clean['tweet_text'].apply(lambda x: re.sub(r'@mention', '', x))
# remove {link}
df_clean['tweet_text'] = df_clean['tweet_text'].apply(lambda x: re.sub(r'{link}', '', x))
# remove &text; html chars
df_clean['tweet_text'] = df_clean['tweet_text'].apply(lambda x: re.sub(r'&[a-z]+;', '', x))
# [video]
df_clean['tweet_text'] = df_clean['tweet_text'].apply(lambda x: re.sub(r"\[video\]", '', x))
# remove all remaining characters that aren't letters, white space, or 
# the following #:)(/\='] that are used in emojis or hashtags
df_clean['tweet_text'] = df_clean['tweet_text'].apply(lambda x: re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", '', x))
df_clean

<ipython-input-84-89e914d04461>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['tweet_text'] = df_clean['tweet_str'].str.lower()
<ipython-input-84-89e914d04461>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['tweet_text'] = df_clean['tweet_text'].apply(lambda x: re.sub(r'https?:\/\/\S+', '', x))
<ipython-input-84-89e914d04461>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

,county,original_tweet,tweets_list,date_tweeted,tweet_str,tweet_slice,tweet_text
0,Texas test,[@tigerfan1975lsu @AmericanAir Thanks @America...,[@tigerfan1975lsu @AmericanAir Thanks @America...,3/18/21,['@tigerfan1975lsu @AmericanAir Thanks @Americ...,r ! Anyway we can get a badge or something to ...,'tigerfanlsu americanair thanks americanair a...
0,Texas test,[What are some of the top compliance &amp; eth...,[@tigerfan1975lsu @AmericanAir Thanks @America...,3/17/21,['What are some of the top compliance &amp; et...,stories from the past week? In the only weekl...,'what are some of the top compliance ethics s...
0,Texas test,[What are some of the top compliance &amp; eth...,[@tigerfan1975lsu @AmericanAir Thanks @America...,3/16/21,['What are some of the top compliance &amp; et...,stories from the past week? In the only weekl...,'what are some of the top compliance ethics s...
0,Texas test,[@ITHERETWEETER1 @katyekellyeand1 @MySwilly @S...,[@tigerfan1975lsu @AmericanAir Thanks @America...,3/16/21,['@ITHERETWEETER1 @katyekellyeand1 @MySwilly @...,KLORE1 @hellsoasis @DitseaYella @Promethiumban...,'itheretweeter katyekellyeand myswilly shocklo...
0,Texas test,[@DonaldJTrumpJr Please thank your dad for me ...,[@tigerfan1975lsu @AmericanAir Thanks @America...,3/16/21,"[""@DonaldJTrumpJr Please thank your dad for me...",ce he is not here on twitter! Being a retired ...,donaldjtrumpjr please thank your dad for me si...
...,...,...,...,...,...,...,...
23,Johnson County KS,[@thomaskaine5 It covered what needed to be co...,[And not a mask in sight....#PuntaCana #MaskFr...,3/12/21,['@thomaskaine5 It covered what needed to be c...,"ed. Sadly, I feel like he is preaching to the...",'thomaskaine it covered what needed to be cove...
24,King County WA,[@Hilton @HiltonHotels I see Hilton properties...,[@Hilton @HiltonHotels I see Hilton properties...,3/18/21,['@Hilton @HiltonHotels I see Hilton propertie...,Idaho requiring guests to wear masks while Ma...,'hilton hiltonhotels i see hilton properties i...
24,King County WA,[I never say anything to my cousin anymore bc ...,[@Hilton @HiltonHotels I see Hilton properties...,3/16/21,['I never say anything to my cousin anymore bc...,is the only place she hasn’t blocked me and I ...,'i never say anything to my cousin anymore bc ...
24,King County WA,[@offgridteacher I’ve eaten indoors but the ma...,[@Hilton @HiltonHotels I see Hilton properties...,3/13/21,['@offgridteacher I’ve eaten indoors but the m...,requirements are pretty strict (masks to enter...,'offgridteacher ive eaten indoors but the mask...


## Do a spot check to make sure that the data has been cleaned well:

In [85]:
df_clean['tweet_text'].iloc[5]

"'raw story: fully vaccinated but scared to remove your mask experts say getting back to a mask-free norm may take time for some\\n googlenews' '-mar-']"

In [86]:
twitter_drop['tweet_str'].iloc[5]

"['Raw Story: Fully vaccinated but scared to remove your mask? Experts say getting back to a mask-free norm may take time for some.\\nhttps://t.co/v3pZrjIMGS\\n\\nvia @GoogleNews', '15-Mar-2021']"

# Save as csv

In [87]:
df_clean.to_csv('cleaned7_tweets.csv')

#########################################################################################################